# FINAL PROJECT PHASE II

## Research Question:

### Question:

+explain in detail

In [139]:
#import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.linear_model import LinearRegression, LogisticRegression
import duckdb

## Data Overview:

+description

In [140]:
#import packages and read csv file
df=pd.read_csv("/Users/ashleychiang/2950PROJECT/Amazon Customer Behavior Survey.csv",thousands=",")

print(df.shape)
df.head()

(602, 23)


,Timestamp,age,Gender,Purchase_Frequency,Purchase_Categories,Personalized_Recommendation_Frequency,Browsing_Frequency,Product_Search_Method,Search_Result_Exploration,Customer_Reviews_Importance,...,Saveforlater_Frequency,Review_Left,Review_Reliability,Review_Helpfulness,Personalized_Recommendation_Frequency,Recommendation_Helpfulness,Rating_Accuracy,Shopping_Satisfaction,Service_Appreciation,Improvement_Areas
0,2023/06/04 1:28:19 PM GMT+5:30,23,Female,Few times a month,Beauty and Personal Care,Yes,Few times a week,Keyword,Multiple pages,1,...,Sometimes,Yes,Occasionally,Yes,2,Yes,1,1,Competitive prices,Reducing packaging waste
1,2023/06/04 2:30:44 PM GMT+5:30,23,Female,Once a month,Clothing and Fashion,Yes,Few times a month,Keyword,Multiple pages,1,...,Rarely,No,Heavily,Yes,2,Sometimes,3,2,Wide product selection,Reducing packaging waste
2,2023/06/04 5:04:56 PM GMT+5:30,24,Prefer not to say,Few times a month,Groceries and Gourmet Food;Clothing and Fashion,No,Few times a month,Keyword,Multiple pages,2,...,Rarely,No,Occasionally,No,4,No,3,3,Competitive prices,Product quality and accuracy
3,2023/06/04 5:13:00 PM GMT+5:30,24,Female,Once a month,Beauty and Personal Care;Clothing and Fashion;...,Sometimes,Few times a month,Keyword,First page,5,...,Sometimes,Yes,Heavily,Yes,3,Sometimes,3,4,Competitive prices,Product quality and accuracy
4,2023/06/04 5:28:06 PM GMT+5:30,22,Female,Less than once a month,Beauty and Personal Care;Clothing and Fashion,Yes,Few times a month,Filter,Multiple pages,1,...,Rarely,No,Heavily,Yes,4,Yes,2,2,Competitive prices,Product quality and accuracy


In [141]:
#we created a new dataframe that is relevant to our research question.

new_df= duckdb.sql('''

SELECT age, Review_Helpfulness, Recommendation_Helpfulness, 
Personalized_Recommendation_Frequency, Cart_Completion_Frequency, 
Add_to_Cart_Browsing, Customer_Reviews_Importance
FROM df

''').df()

new_df.head()

,age,Review_Helpfulness,Recommendation_Helpfulness,Personalized_Recommendation_Frequency,Cart_Completion_Frequency,Add_to_Cart_Browsing,Customer_Reviews_Importance
0,23,Yes,Yes,Yes,Sometimes,Yes,1
1,23,Yes,Sometimes,Yes,Often,Yes,1
2,24,No,No,No,Sometimes,Yes,2
3,24,Yes,Sometimes,Sometimes,Sometimes,Maybe,5
4,22,Yes,Yes,Yes,Sometimes,Yes,1


In [142]:
#We changed the values from string to numbers so that we can actually measure the correlation. 


new_df['New_Review_Helpfulness']=new_df['Review_Helpfulness'].\
map({'No': 0, 'Sometimes': 1, 'Yes': 2})
new_df.head()

new_df['New_Recommendation_Helpfulness']=new_df['Recommendation_Helpfulness'].\
map({'No': 0, 'Sometimes': 1, 'Yes': 2})
new_df.head()

new_df['New_Personalized_Recommendation_Frequency']=new_df['Personalized_Recommendation_Frequency'].\
map({'No': 0, 'Sometimes': 1, 'Yes': 2})
new_df.head()

new_df['New_Cart_Completion_Frequency']=new_df['Cart_Completion_Frequency'].\
map({'Never': 0, 'Rarely': 1, 'Sometimes': 2, 'Often':3, 'Always':4})
new_df.head()

new_df['New_Add_to_Cart_Browsing']=new_df['Add_to_Cart_Browsing'].\
map({'No': 0, 'Maybe': 1, 'Yes': 2})
new_df.head()


,age,Review_Helpfulness,Recommendation_Helpfulness,Personalized_Recommendation_Frequency,Cart_Completion_Frequency,Add_to_Cart_Browsing,Customer_Reviews_Importance,New_Review_Helpfulness,New_Recommendation_Helpfulness,New_Personalized_Recommendation_Frequency,New_Cart_Completion_Frequency,New_Add_to_Cart_Browsing
0,23,Yes,Yes,Yes,Sometimes,Yes,1,2,2,2,2,2
1,23,Yes,Sometimes,Yes,Often,Yes,1,2,1,2,3,2
2,24,No,No,No,Sometimes,Yes,2,0,0,0,2,2
3,24,Yes,Sometimes,Sometimes,Sometimes,Maybe,5,2,1,1,2,1
4,22,Yes,Yes,Yes,Sometimes,Yes,1,2,2,2,2,2


In [143]:
#We dropped the old columns that weren't relevant anymore as we got a new column. 


new_new_df=duckdb.sql('''
SELECT age, New_Review_Helpfulness, New_Recommendation_Helpfulness, 
New_Personalized_Recommendation_Frequency, New_Cart_Completion_Frequency, 
New_Add_to_Cart_Browsing, Customer_Reviews_Importance
FROM new_df
'''                  
).df()
new_df.head()



,age,Review_Helpfulness,Recommendation_Helpfulness,Personalized_Recommendation_Frequency,Cart_Completion_Frequency,Add_to_Cart_Browsing,Customer_Reviews_Importance,New_Review_Helpfulness,New_Recommendation_Helpfulness,New_Personalized_Recommendation_Frequency,New_Cart_Completion_Frequency,New_Add_to_Cart_Browsing
0,23,Yes,Yes,Yes,Sometimes,Yes,1,2,2,2,2,2
1,23,Yes,Sometimes,Yes,Often,Yes,1,2,1,2,3,2
2,24,No,No,No,Sometimes,Yes,2,0,0,0,2,2
3,24,Yes,Sometimes,Sometimes,Sometimes,Maybe,5,2,1,1,2,1
4,22,Yes,Yes,Yes,Sometimes,Yes,1,2,2,2,2,2


In [144]:
#We looked for outliers for age. We know that if age is under 13, they can't make accounts to buy things. 

real_df=duckdb.sql('''
SELECT *
FROM new_new_df
WHERE age>=13
''').df()

real_df.head()

,age,New_Review_Helpfulness,New_Recommendation_Helpfulness,New_Personalized_Recommendation_Frequency,New_Cart_Completion_Frequency,New_Add_to_Cart_Browsing,Customer_Reviews_Importance
0,23,2,2,2,2,2,1
1,23,2,1,2,3,2,1
2,24,0,0,0,2,2,2
3,24,2,1,1,2,1,5
4,22,2,2,2,2,2,1


In [145]:
#We want to categorize into 5 different age groups.

real_df.loc[(real_df["age"] >= 13) & (real_df["age"] <= 18), "age"] = 1
real_df.loc[(real_df["age"] >= 19) & (real_df["age"] <= 25), "age"] = 2
real_df.loc[(real_df["age"] >= 26) & (real_df["age"] <= 40), "age"] = 3
real_df.loc[(real_df["age"] >= 41) & (real_df["age"] <= 64), "age"] = 4
real_df.loc[(real_df["age"] >= 65), "age"] = 5

In [146]:
# average of each column

final_df = real_df.groupby('age')[['New_Review_Helpfulness', 'New_Recommendation_Helpfulness', 'New_Personalized_Recommendation_Frequency', 'New_Cart_Completion_Frequency', 'New_Add_to_Cart_Browsing', 'Customer_Reviews_Importance']].mean()
final_df.head()

,New_Review_Helpfulness,New_Recommendation_Helpfulness,New_Personalized_Recommendation_Frequency,New_Cart_Completion_Frequency,New_Add_to_Cart_Browsing,Customer_Reviews_Importance
age,,,,,,
1,0.888889,0.777778,0.444444,2.277778,0.833333,2.500000
2,1.317647,1.031373,0.780392,2.176471,1.278431,2.407843
3,1.077982,0.908257,0.779817,2.293578,1.059633,2.490826
4,1.018692,1.000000,0.859813,2.214953,0.990654,2.616822
5,1.000000,1.500000,1.000000,2.500000,0.500000,2.500000


## Data Description:

### Motivation:


### Data Source:

## Data Limitation:

## Explorative Analysis:

## Questions for Reviewer: